# Image Variant Generation with S3 Storage

This notebook:
1. Queries a database for deal IDs
2. Generates variant images using OpenAI
3. Stores images in S3
4. Implements async processing for efficiency

In [31]:
import pandas as pd
import os
import psycopg2
from dotenv import load_dotenv
import boto3
import time
import base64
import asyncio
import aiohttp
import tempfile
import sys
from botocore.exceptions import NoCredentialsError
from concurrent.futures import ThreadPoolExecutor
import subprocess
import json
from io import BytesIO
# Load environment variables
load_dotenv()
# Configure AWS credentials
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
)

## Query Database for Deal IDs

Execute SQL to get deal information including:
- deal_voucher_id
- original_image_id
- variant_image_id
- batch_name
- enter_test_ts
- exit_test_ts
- open_ai_prompt

In [29]:
def get_deals_for_processing():
    # Establish connection to Redshift
    conn = psycopg2.connect(
        host=os.environ.get("REDSHIFT_HOST" ),
        port=os.environ.get("REDSHIFT_PORT"),
        dbname=os.environ.get("REDSHIFT_DBNAME"),
        user=os.environ.get("REDSHIFT_USER"),
        password=os.environ.get("REDSHIFT_PASSWORD")
    )
    
    # Example query - modify as needed
    query = """
WITH visitors AS (
    SELECT
        deal_id_evar,
        COUNT(DISTINCT visitor_id) AS visitors
    FROM real.omniture_events
    WHERE trunc(date_time) >= trunc(sysdate) - 7
      AND product = 'wowdtm'
      AND (
            url_evar LIKE '%/deal/%' OR
            url_evar LIKE '%/e/%' OR
            url_evar LIKE '%/email-deals/%'
          )
    GROUP BY deal_id_evar
)
SELECT
    CAST(dv.id AS INTEGER) AS id,
    CASE 
        WHEN dvc.canonical_path_type = 'NATIONAL' THEN dv.email_subject 
        ELSE 
            CASE 
                WHEN dvc.canonical_path_type = 'LOCAL' THEN dv.deal_product 
            END 
    END AS email_subject,
    dvc.name AS category_name,
    dvc.canonical_path_type as vertical,
    dvsc.name AS sub_category_name,
    CAST(COALESCE(v.visitors, 0) AS INTEGER) AS visitors_last_7_days,
    CAST(rank() OVER (ORDER BY COALESCE(v.visitors, 0) DESC) AS INTEGER) AS visitor_rank,
    CAST(dvi.id AS INTEGER) AS image_id_pos_0,
    'https://static.wowcher.co.uk/images/deal/' || dvi.deal_voucher_id || '/' || dvi.id || '.' || dvi.extension AS image_url_pos_0,
    dvi.extension
FROM real.deal_voucher dv
left JOIN real.product p ON p.id = dv.id AND p.status_id = 1
JOIN visitors v ON v.deal_id_evar = dv.id
LEFT JOIN real.deal_voucher_site dvs ON dvs.deal_voucher_id = dv.id
LEFT JOIN real.deal_voucher_image dvi ON dvi.deal_voucher_id = dv.id AND dvi.position = 0
LEFT JOIN real.deal_voucher_category dvc ON dvc.id = dv.category_id
LEFT JOIN real.deal_voucher_sub_category dvsc ON dvsc.id = dv.sub_category_id
LEFT JOIN real.site s ON s.id = dv.deal_location_id AND s.site_name = 'National Deal'
WHERE trunc(dv.closing_date) >= trunc(sysdate) + 21
AND dv.currency = 'GBP'
AND NOT EXISTS (
    SELECT 1
      FROM temp.opt_image_variants oiv
      WHERE oiv.deal_voucher_id = dv.id
      AND (
      ((batch_name ILIKE '%manual%' AND status IN (1,3,5))
        or (batch_name = 'OPEN AI Images' AND status IN (1,3))
       OR (batch_name NOT IN ('Manual Opt', 'OPEN AI Images') AND status = 1)
        )
    )
)
AND dvc.canonical_path_type = 'LOCAL'
GROUP BY dv.id, dv.email_subject, dvc.name, dvsc.name,dvc.canonical_path_type, dvi.id, dvi.deal_voucher_id, dvi.extension, v.visitors,dv.deal_product
ORDER BY COALESCE(v.visitors, 0) DESC
LIMIT 2000;
    """
    df = pd.read_sql(query, conn)
    conn.close()
    return df
# Get deals to process
deals_df = get_deals_for_processing()
deals_df.head()

/var/folders/dk/zpttw0wn0192zryb4bdz19lr0000gn/T/ipykernel_25673/3666989610.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,id,email_subject,category_name,vertical,sub_category_name,visitors_last_7_days,visitor_rank,image_id_pos_0,image_url_pos_0,extension
0,31167175,500 EuroMillions Lines & 500 Millionaires Raff...,Entertainment,LOCAL,Bingo & Gambling,40650,1,1646133,https://static.wowcher.co.uk/images/deal/31167...,jpg
1,40047459,PRICE DROP Bannatyne Pamper Spa Day: 3 Treatme...,Beauty,LOCAL,Spa,17373,2,1593524,https://static.wowcher.co.uk/images/deal/40047...,jpg
2,39394863,Bannatyne Health Club & Spa Day & Lunch for 2:...,Beauty,LOCAL,Spa,11075,3,1555151,https://static.wowcher.co.uk/images/deal/39394...,jpg
3,40167584,TGI Fridays: Two Course Dining & Cocktails for...,Restaurants & Bars,LOCAL,Grill/BBQ,10921,4,1598318,https://static.wowcher.co.uk/images/deal/40167...,jpg
4,39431357,Toby Carvery 2 Course Dining for 2 People - Fa...,Restaurants & Bars,LOCAL,British,7953,5,1559121,https://static.wowcher.co.uk/images/deal/39431...,jpg


In [30]:
deals_df = deals_df.head(250)
deals_df

,id,email_subject,category_name,vertical,sub_category_name,visitors_last_7_days,visitor_rank,image_id_pos_0,image_url_pos_0,extension
0,31167175,500 EuroMillions Lines & 500 Millionaires Raff...,Entertainment,LOCAL,Bingo & Gambling,40650,1,1646133,https://static.wowcher.co.uk/images/deal/31167...,jpg
1,40047459,PRICE DROP Bannatyne Pamper Spa Day: 3 Treatme...,Beauty,LOCAL,Spa,17373,2,1593524,https://static.wowcher.co.uk/images/deal/40047...,jpg
2,39394863,Bannatyne Health Club & Spa Day & Lunch for 2:...,Beauty,LOCAL,Spa,11075,3,1555151,https://static.wowcher.co.uk/images/deal/39394...,jpg
3,40167584,TGI Fridays: Two Course Dining & Cocktails for...,Restaurants & Bars,LOCAL,Grill/BBQ,10921,4,1598318,https://static.wowcher.co.uk/images/deal/40167...,jpg
4,39431357,Toby Carvery 2 Course Dining for 2 People - Fa...,Restaurants & Bars,LOCAL,British,7953,5,1559121,https://static.wowcher.co.uk/images/deal/39431...,jpg
...,...,...,...,...,...,...,...,...,...,...
245,40653309,UNESCO World Heritage Canoe Aqueduct Tour for ...,Activities,LOCAL,Outdoor/Action,213,246,1639586,https://static.wowcher.co.uk/images/deal/40653...,jpg
246,39606452,Simply Italian Cooking Class with 2 Drinks & £...,Activities,LOCAL,Other,213,246,1574422,https://static.wowcher.co.uk/images/deal/39606...,jpg
247,40567159,4* Mother & Daughter ELEMIS Country House Spa ...,Beauty,LOCAL,Spa,213,246,1639294,https://static.wowcher.co.uk/images/deal/40567...,jpg
248,24734055,Dining for 2 - Experience Gift Pack,Restaurants & Bars,LOCAL,British,209,249,1509561,https://static.wowcher.co.uk/images/deal/24734...,jpg


## S3 Upload Functions

Functions to upload generated images to S3

In [10]:
def upload_to_s3(file_content, bucket_name, s3_key):
    """
    Upload a file to S3
    
    Parameters:
    - file_content: Binary content of the file
    - bucket_name: S3 bucket name
    - s3_key: Path in S3 where file will be stored
    
    Returns:
    - URL of the uploaded file
    """
    try:
        # Determine content type based on file extension
        extension = os.path.splitext(s3_key)[1].lower()
        content_type = 'image/jpeg' if extension in ['.jpg', '.jpeg'] else \
                      'image/png' if extension == '.png' else \
                      'image/webp' if extension == '.webp' else \
                      'application/octet-stream'
                      
        s3_client.put_object(
            Body=file_content,
            Bucket=bucket_name,
            Key=s3_key,
            ContentType=content_type
        )
        return f"https://static.wowcher.co.uk/{s3_key}"
    except NoCredentialsError:
        print("Credentials not available")
        return None

In [28]:
deals_df

,id,email_subject,category_name,vertical,sub_category_name,visitors_last_7_days,visitor_rank,image_id_pos_0,image_url_pos_0,extension
0,31167175,None,Entertainment,LOCAL,Bingo & Gambling,40476,1,1646133,https://static.wowcher.co.uk/images/deal/31167...,jpg
1,40047459,None,Beauty,LOCAL,Spa,17128,2,1593524,https://static.wowcher.co.uk/images/deal/40047...,jpg
2,39394863,None,Beauty,LOCAL,Spa,10939,3,1555151,https://static.wowcher.co.uk/images/deal/39394...,jpg
3,40167584,None,Restaurants & Bars,LOCAL,Grill/BBQ,10764,4,1598318,https://static.wowcher.co.uk/images/deal/40167...,jpg
4,39431357,None,Restaurants & Bars,LOCAL,British,7832,5,1559121,https://static.wowcher.co.uk/images/deal/39431...,jpg
...,...,...,...,...,...,...,...,...,...,...
245,39606452,None,Activities,LOCAL,Other,211,245,1574422,https://static.wowcher.co.uk/images/deal/39606...,jpg
246,30065007,None,Beauty,LOCAL,Spa,211,245,1227766,https://static.wowcher.co.uk/images/deal/30065...,jpg
247,40567159,None,Beauty,LOCAL,Spa,210,248,1639294,https://static.wowcher.co.uk/images/deal/40567...,jpg
248,39791439,None,Restaurants & Bars,LOCAL,Gourmet,205,249,1597995,https://static.wowcher.co.uk/images/deal/39791...,jpg


## Image Generation

Function to call the generate_image.py script and process the results

In [ ]:
# Cell to replace external script dependency with integrated functionality
import pandas as pd
import os
import psycopg2
from dotenv import load_dotenv
import requests
import base64
from openai import OpenAI
from io import BytesIO

# Load environment variables if not already done
if 'client' not in locals():
    load_dotenv()
    client = OpenAI(api_key=os.getenv('OPEN_AI_API_KEY'))
    print(f"OpenAI client initialized with API key: {os.getenv('OPEN_AI_API_KEY')[:5]}...")

def get_deal_data_for_image(deal_id, vertical):
    """Get deal data needed for image generation"""
    # Establish connection to Redshift
    conn = psycopg2.connect(
        host=os.environ.get("REDSHIFT_HOST"),
        port=os.environ.get("REDSHIFT_PORT"),
        dbname=os.environ.get("REDSHIFT_DBNAME"),
        user=os.environ.get("REDSHIFT_USER"),
        password=os.environ.get("REDSHIFT_PASSWORD")
    )

    # Get email subject
    email_subject_query = """
    SELECT email_subject 
    FROM wowdwhprod.real.deal_voucher
    WHERE id = %s
    """
    with conn.cursor() as cur:
        cur.execute(email_subject_query, (deal_id,))
        email_subject_result = cur.fetchone()
        email_subject = email_subject_result[0] if email_subject_result else "Deal"

    # Get image URLs and extract extension information
    image_query = """
    SELECT 
        'https://static.wowcher.co.uk/images/deal/' || deal_voucher_id || '/' || id || '.' || extension AS image_url,
        extension
    FROM wowdwhprod.real.deal_voucher_image
    WHERE deal_voucher_id = %s
    ORDER BY position
    LIMIT 10
    """
    with conn.cursor() as cur:
        cur.execute(image_query, (deal_id,))
        image_results = cur.fetchall()
        image_urls = [row[0] for row in image_results]
        extensions = [row[1] for row in image_results]
        original_extension = extensions[0] if extensions else "png"

    # Get highlights
    highlights_query = """
    SELECT
    SPLIT_PART(highlight, ':', 1) AS highlight
    FROM wowdwhprod.real.deal_voucher_highlight
    WHERE deal_voucher_id = %s
    LIMIT 3;

    """
    with conn.cursor() as cur:
        cur.execute(highlights_query, (deal_id,))
        highlights_results = cur.fetchall()
        highlights = [row[0] for row in highlights_results]

    conn.close()

    # Build the prompt
    formatted_highlights = "\n".join([f"• {h}" for h in highlights]) if highlights else ""
    national_prompt = f"""
    Create ONE high-resolution photo-realistic image advertising **{email_subject}**.

    Final image must contain **zero spelling mistakes**.  

    1. **Source images** – You have multiple angles.  
    • Accurately represent the product; do **not** invent new colours or features.  
    • If colour variants exist, PICK ONE colour and keep it consistent, though you should highlight the colours available in the add or the fact that there are multiple colours.
    - do not put any prices in the image. 

    2. **Scene & background**  
    • Place the product in a realistic, aspirational environment that makes sense for its use.  
    • Adjust lighting and depth of field so the product is the clear focal point.  
    • Background must not overpower or obscure the product.

    3. **Infographic & text elements**  
        • Do **not** repeat the headline anywhere else in the artwork.  
        • Any additional text or graphics must be limited to the 2-4 call-outs listed below.
        - Try and place the additional text or stickers on the left of the image. 
    • Overlay 2-4 concise call-outs drawn from these highlights:  
        {formatted_highlights}  
    • Position all call-outs **outside** the bottom-right 20% of the frame.

    4. **Design constraints**  
    • Keep bottom-right area completely free of any graphics or text. 
    • Maintain 4 px padding around all text boxes.  
    • No brand logos unless provided in the source images.

    """
    
    local_prompt = f""" 
    Create ONE high-resolution, photo-realistic promotional image advertising {email_subject}. The Image should look like it was taken by a proffessional phographer, well lit and using a proffessional camera. Dont put the whole text from the product name in the image, condense it. 

    Final image must contain zero spelling mistakes.

    1. Service Representation
    Visually communicate the core experience or service in an authentic and aspirational way.
    Include people only if appropriate — expressions must look natural, relaxed, and genuinely engaged. The people should be objectively attractive.
    Avoid exaggerated or artificial staging.
    Focus on one setting or moment that clearly represents the value of the experience. 
    Make sure any people providing the service are dressed in a proffesional manner. 
    Make sure any people's bodies in the image are positioned in a way that is not physically possible. 
    If applicable, include accurate tools, furnishings, or attire that reflect the service.

    2. Scene & Background
    Place the experience in a realistic, appealing setting suited to the type of service.
    Use natural lighting and soft depth of field to keep the service or activity as the clear focal point.
    The background should feel aspirational but believable — it must not compete with or overshadow the subject.
    Avoid busy or generic backdrops; choose settings that suggest quality, comfort, or excitement.

    3. Infographic & Text Elements
    Add title text towards the top of the image.
    Do not repeat the same text elsewhere in the image.
    Ensure all text and stickers avoid the bottom-right 20% of the image.
    Choose clear, legible text with 4px padding around all elements.

    4. Design Constraints
    Keep the bottom-right corner completely clear of any graphics or text.
    Do not include any pricing or logos unless specifically provided.
    Ensure realism — no invented props, features, or environments.
    Colour palettes, uniforms, and tools must accurately reflect the service being portrayed.

        
    """
    
    prompt = national_prompt if vertical == "NATIONAL" else local_prompt
    
    return {
        'prompt': prompt,
        'image_urls': image_urls,
        'original_extension': original_extension
    }

def download_image_to_file(url, filename):
    """Download an image from URL and save to file"""
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        return filename
    else:
        raise Exception(f"Failed to download image from {url}")
# Add this to your imports
from tqdm.notebook import tqdm
import urllib.request
import logging

# Set up logging to control verbosity
logging.basicConfig(level=logging.WARNING)  # Set to WARNING to hide INFO and DEBUG messages

def generate_image_integrated(deal_id, original_id, temp_dir, vertical, verbose=False):
    """
    Generate image using OpenAI's API with minimal output
    """
    try:
        if verbose:
            print(f"Processing deal {deal_id}")
        
        # Get data for the deal
        deal_data = get_deal_data_for_image(deal_id, vertical)
        prompt = deal_data['prompt']
        image_urls = deal_data['image_urls']
        original_extension = deal_data['original_extension']
        
        # Create output filename
        output_filename = os.path.join(temp_dir, f"variant_{deal_id}_{original_id}.{original_extension}")
        
        if not image_urls:
            raise Exception("No images found for this deal")
        
        # Download images silently
        image_files = []
        temp_filenames = []
        for idx, url in enumerate(image_urls[:16]):
            temp_filename = os.path.join(temp_dir, f"temp_image_{deal_id}_{idx}.png")
            download_image_to_file(url, temp_filename)
            temp_filenames.append(temp_filename)
            image_files.append(open(temp_filename, "rb"))
        
        # Call OpenAI API
        result = client.images.edit(
            model="gpt-image-1",
            image=image_files,
            prompt=prompt,
            size="1536x1024",
            quality="high",
            background="auto",
            n=1,
            moderation = "low"
        )
        
        # Process and save the response
        image_base64 = result.data[0].b64_json
        image_bytes = base64.b64decode(image_base64)
        with open(output_filename, "wb") as f:
            f.write(image_bytes)
            
        # Close file handles
        for f in image_files:
            f.close()
            
        # Delete temporary files
        for filename in temp_filenames:
            if os.path.exists(filename):
                try:
                    os.remove(filename)
                except:
                    pass
        
        # Process token usage details silently
        token_info = {}
        if hasattr(result, 'usage'):
            total_tokens = result.usage.total_tokens
            input_tokens = result.usage.input_tokens
            output_tokens = result.usage.output_tokens
            input_text_tokens = result.usage.input_tokens_details.text_tokens
            input_image_tokens = result.usage.input_tokens_details.image_tokens
            
            token_info["Total tokens"] = str(total_tokens)
            token_info["Input tokens"] = str(input_tokens)
            token_info["Output tokens"] = str(output_tokens)
            token_info["Input text tokens"] = str(input_text_tokens)
            token_info["Input image tokens"] = str(input_image_tokens)
            
            # Calculate cost
            cost = (input_text_tokens * 5 + input_image_tokens * 10 + output_tokens * 40) / 1000000
            token_info["Cost"] = f"${cost:.6f}"
            
            if verbose:
                print(f"Cost: ${cost:.6f}")
        
        return output_filename, original_extension, token_info
        
    except Exception as e:
        if verbose:
            print(f"Error generating image for deal {deal_id}: {str(e)}")
        return None, None, None

async def process_deals_async(deals_df, max_workers=4, verbose=False):
    """
    Process multiple deals asynchronously with clean, minimal output
    """
    results = []
    
    if verbose:
        print(f"Available columns in dataframe: {list(deals_df.columns)}")
    
    # Create a temporary directory for image files
    with tempfile.TemporaryDirectory() as temp_dir:
        # Use ThreadPoolExecutor for parallelization
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Create tasks for all deals
            futures = []
            for idx, row in deals_df.iterrows():
                # Get deal_id, original_id, and vertical from the actual column names
                deal_id = row.get('id')
                original_id = row.get('image_id_pos_0', 'main')
                vertical = row.get('vertical', 'LOCAL')  # Default to LOCAL if not specified
                    
                if deal_id is None:
                    if verbose:
                        print(f"Warning: Could not find deal ID in row: {row}")
                    continue
                    
                future = executor.submit(generate_image_integrated, deal_id, original_id, temp_dir, vertical, verbose)
                futures.append((future, deal_id, original_id, row))
            
            # Create just ONE overall progress bar
            print(f"Processing {len(futures)} deals...")
            progress_bar = tqdm(total=len(futures), desc="Overall progress")
            
            # Process results as they complete
            for future, deal_id, original_id, row in futures:
                try:
                    image_path, extension, token_info = future.result()
                    
                    if image_path:
                        # Read the image file
                        with open(image_path, 'rb') as img_file:
                            img_content = img_file.read()
                        
                        # Upload to S3 with correct extension
                        s3_key = f"images/deal/{deal_id}/{original_id}_variant.{extension}"
                        s3_url = upload_to_s3(img_content, 'static.wowcher.co.uk', s3_key)
                        
                        # Add to results
                        result_row = row.to_dict()
                        result_row.update({
                            'status': 'success',
                            's3_url': s3_url,
                            'token_info': token_info,
                            'extension': extension,
                            'processed_timestamp': pd.Timestamp.now()
                        })
                        results.append(result_row)
                    else:
                        # Add failure to results
                        result_row = row.to_dict()
                        result_row.update({
                            'status': 'failed',
                            'error': 'Image generation failed',
                            'processed_timestamp': pd.Timestamp.now()
                        })
                        results.append(result_row)
                        
                except Exception as e:
                    if verbose:
                        print(f"Error processing deal {deal_id}: {str(e)}")
                    result_row = row.to_dict()
                    result_row.update({
                        'status': 'failed',
                        'error': str(e),
                        'processed_timestamp': pd.Timestamp.now()
                    })
                    results.append(result_row)
                
                # Update progress bar
                progress_bar.update(1)
            
            # Close progress bar
            progress_bar.close()
    
    return pd.DataFrame(results)

## Run the Process

Execute the async processing and display results

In [22]:
deals_sample = deals_df

print(f"Selected {len(deals_sample)} deals")

# Process deals asynchronously
results_df = await process_deals_async(deals_sample, max_workers=50, verbose=False)

# Filter to get only successful results (the "winners")
winners_df = results_df[results_df['status'] == 'success'].copy()
print(f"\nSuccessful generations: {len(winners_df)} out of {len(results_df)}")

# Display the winners
display(winners_df)

# Calculate total cost for successful generations
if 'token_info' in winners_df.columns:
    total_cost = 0.0
    for _, row in winners_df.iterrows():
        if 'token_info' in row and row['token_info'] and 'Cost' in row['token_info']:
            cost_str = row['token_info']['Cost'].replace('$', '')
            try:
                total_cost += float(cost_str)
            except:
                pass
    print(f"Total cost for successful generations: ${total_cost:.4f}")





Selected 250 deals
Processing 250 deals...


Overall progress:   0%|          | 0/250 [00:00<?, ?it/s]


Successful generations: 208 out of 250


,id,email_subject,category_name,vertical,sub_category_name,visitors_last_7_days,visitor_rank,image_id_pos_0,image_url_pos_0,extension,status,error,processed_timestamp,s3_url,token_info
1,40047459,None,Beauty,LOCAL,Spa,17128,2,1593524,https://static.wowcher.co.uk/images/deal/40047...,jpg,success,NaN,2025-06-11 15:11:06.072274,https://static.wowcher.co.uk/images/deal/40047...,"{'Total tokens': '10122', 'Input tokens': '391..."
2,39394863,None,Beauty,LOCAL,Spa,10939,3,1555151,https://static.wowcher.co.uk/images/deal/39394...,jpg,success,NaN,2025-06-11 15:11:07.065915,https://static.wowcher.co.uk/images/deal/39394...,"{'Total tokens': '10125', 'Input tokens': '391..."
3,40167584,None,Restaurants & Bars,LOCAL,Grill/BBQ,10764,4,1598318,https://static.wowcher.co.uk/images/deal/40167...,jpg,success,NaN,2025-06-11 15:11:12.825978,https://static.wowcher.co.uk/images/deal/40167...,"{'Total tokens': '10125', 'Input tokens': '391..."
4,39431357,None,Restaurants & Bars,LOCAL,British,7832,5,1559121,https://static.wowcher.co.uk/images/deal/39431...,jpg,success,NaN,2025-06-11 15:11:14.300069,https://static.wowcher.co.uk/images/deal/39431...,"{'Total tokens': '10120', 'Input tokens': '391..."
5,23021958,None,Activities,LOCAL,Motoring,7148,6,895591,https://static.wowcher.co.uk/images/deal/23021...,jpg,success,NaN,2025-06-11 15:11:14.504331,https://static.wowcher.co.uk/images/deal/23021...,"{'Total tokens': '8025', 'Input tokens': '1817..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,40653309,None,Activities,LOCAL,Outdoor/Action,211,245,1639586,https://static.wowcher.co.uk/images/deal/40653...,jpg,success,NaN,2025-06-11 15:17:32.052211,https://static.wowcher.co.uk/images/deal/40653...,"{'Total tokens': '8724', 'Input tokens': '2516..."
245,39606452,None,Activities,LOCAL,Other,211,245,1574422,https://static.wowcher.co.uk/images/deal/39606...,jpg,success,NaN,2025-06-11 15:17:32.260848,https://static.wowcher.co.uk/images/deal/39606...,"{'Total tokens': '8376', 'Input tokens': '2168..."
246,30065007,None,Beauty,LOCAL,Spa,211,245,1227766,https://static.wowcher.co.uk/images/deal/30065...,jpg,success,NaN,2025-06-11 15:18:30.509078,https://static.wowcher.co.uk/images/deal/30065...,"{'Total tokens': '9077', 'Input tokens': '2869..."
247,40567159,None,Beauty,LOCAL,Spa,210,248,1639294,https://static.wowcher.co.uk/images/deal/40567...,jpg,success,NaN,2025-06-11 15:18:30.952483,https://static.wowcher.co.uk/images/deal/40567...,"{'Total tokens': '10128', 'Input tokens': '392..."


Total cost for successful generations: $56.3929


### Moving winners into test


In [24]:
status_summary = results_df.groupby(['category_name', 'status']).size().unstack(fill_value=0)
print(status_summary)


status              failed  success
category_name                      
Activities               1       46
Beauty                  37       74
Entertainment            3       20
Fitness                  0        2
Healthcare               0        7
Learning                 0        1
Restaurants & Bars       1       56
Tradesmen                0        2


In [25]:
winners_df.to_csv('first205localImages.csv')